1. 导入数据+查看原始数据信息

In [ ]:
import mne
data_path = r'xxx\BCICIV_2a_gdf\A01T.gdf'
raw = mne.to.read_raw_gdf(data_path, preload=True)
'''
# MNE-Python 中对多种格式的脑电数据都进行了支持：
# *** 如数据后缀为 .set (来自 EEGLAB 的数据)
# 使用 mne.io.read_raw_eeglab()

# *** 如数据后缀为 .vhdr (BrainVision 系统)
# 使用 mne.io.read_raw_brainvison()

# *** 如数据后缀为 .edf
# 使用 mne.io.read_raw_edf()

# *** 如数据后缀为 .bdf (BioSemi放大器)
# 使用 mne.io.read_raw_bdf()

# *** 如数据后缀为 .gdf
# 使用 mne.io.read_raw_gdf()

# *** 如数据后缀为 .cnt (Neuroscan系统)
# 使用 mne.io.read_raw_cnt()

# *** 如数据后缀为 .egi 或 .mff
# 使用 mne.io.read_raw_egi()

# *** 如数据后缀为 .data
# 使用 mne.io.read_raw_nicolet()

# *** 如数据后缀为 .nxe (Nexstim eXimia系统)
# 使用 mne.io.read_raw_eximia()

# *** 如数据后缀为 .lay 或 .dat (Persyst系统)
# 使用 mne.io.read_raw_persyst()

# *** 如数据后缀为 .eeg (Nihon Kohden系统)
# 使用 mne.io.read_raw_nihon()
'''
print(raw)
print(raw.info)

2. 绘制原始数据波形图

In [ ]:
raw.plot(duration=5, n_channels=255, clipping=None)